[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainLGBmodel.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 15.6MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/projects


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

[Errno 2] No such file or directory: '../content/gdrive/My Drive/projects/'
/content/gdrive/My Drive/projects
model_LGB.sav  user_train_wip.csv


In [0]:
s_data = 'user_train_wip.csv'
s_col_tgt = 'totals.transactionRevenue_sum'
l_col_drop = ['date_min', 'date_max',s_col_tgt]
filename = 'model_LGB.sav'

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 10 s, sys: 2.03 s, total: 12.1 s
Wall time: 14.2 s


In [7]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



CPU times: user 1.82 s, sys: 368 ms, total: 2.19 s
Wall time: 2.19 s


In [0]:

params = {
    'learning_rate': 0.03,
    'n_estimators': 2000,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

model = lgb.LGBMRegressor(**params)



In [9]:
%%time
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds=100)

[1]	training's l2: 7.69578	training's rmse: 2.77413	valid_1's l2: 10.7561	valid_1's rmse: 3.27965
Training until validation scores don't improve for 100 rounds.
[2]	training's l2: 7.51179	training's rmse: 2.74076	valid_1's l2: 10.5779	valid_1's rmse: 3.25236
[3]	training's l2: 7.33783	training's rmse: 2.70884	valid_1's l2: 10.4111	valid_1's rmse: 3.22662
[4]	training's l2: 7.17383	training's rmse: 2.6784	valid_1's l2: 10.2509	valid_1's rmse: 3.2017
[5]	training's l2: 7.02014	training's rmse: 2.64956	valid_1's l2: 10.1011	valid_1's rmse: 3.17822
[6]	training's l2: 6.87229	training's rmse: 2.62151	valid_1's l2: 9.95633	valid_1's rmse: 3.15537
[7]	training's l2: 6.73411	training's rmse: 2.59502	valid_1's l2: 9.82306	valid_1's rmse: 3.13418
[8]	training's l2: 6.60543	training's rmse: 2.5701	valid_1's l2: 9.69636	valid_1's rmse: 3.1139
[9]	training's l2: 6.48364	training's rmse: 2.5463	valid_1's l2: 9.57874	valid_1's rmse: 3.09495
[10]	training's l2: 6.3707	training's rmse: 2.52403	valid_1'

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.681,
       importance_type='split', learning_rate=0.03, max_depth=-1,
       min_child_samples=20, min_child_weight=100.0,
       min_split_gain=1.153453257821093e-05, n_estimators=2000, n_jobs=-1,
       num_leaves=128, objective=None, random_state=None,
       reg_alpha=0.0005683292391383383, reg_lambda=1.3896326196366016e-05,
       silent=True, subsample=0.2217, subsample_for_bin=200000,
       subsample_freq=0)

In [10]:
%%time
pickle.dump(model, open(filename, 'wb'))

CPU times: user 417 ms, sys: 7.02 ms, total: 424 ms
Wall time: 1.21 s


In [0]:
files.download(filename) 